In [1]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os

In [2]:
print('Esperamos trabalhar no diretório')
print(os.getcwd())

Esperamos trabalhar no diretório
C:\Users\JUAN\Documents\GitHub\TuneMyMood


In [3]:
df=pd.read_csv("spotifyclassification\data.csv")
df_300=df.head(301)
df_train=pd.read_csv("Trained.csv")

df_train['study'].values.tolist()
df_300.dtypes


Unnamed: 0            int64
acousticness        float64
danceability        float64
duration_ms           int64
energy              float64
instrumentalness    float64
key                   int64
liveness            float64
loudness            float64
mode                  int64
speechiness         float64
tempo               float64
time_signature      float64
valence             float64
target                int64
song_title           object
artist               object
dtype: object

In [4]:
df.head(5)

,Unnamed: 0,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence,target,song_title,artist
0,0,0.0102,0.833,204600,0.434,0.021900,2,0.1650,-8.795,1,0.4310,150.062,4.0,0.286,1,Mask Off,Future
1,1,0.1990,0.743,326933,0.359,0.006110,1,0.1370,-10.401,1,0.0794,160.083,4.0,0.588,1,Redbone,Childish Gambino
2,2,0.0344,0.838,185707,0.412,0.000234,2,0.1590,-7.148,1,0.2890,75.044,4.0,0.173,1,Xanny Family,Future
3,3,0.6040,0.494,199413,0.338,0.510000,5,0.0922,-15.236,1,0.0261,86.468,4.0,0.230,1,Master Of None,Beach House
4,4,0.1800,0.678,392893,0.561,0.512000,5,0.4390,-11.648,0,0.0694,174.004,4.0,0.904,1,Parallel Lines,Junior Boys


In [5]:
# Organizando dados para teste. Todo Speechiness maior que 0.1 será considerado Study=0, menor será Study=1, indicandonque musicas
# com pouco vocal são boas para estudar

df_300['study'] = len(df_300)*[0]
for i in range(150):
    if df_300['speechiness'][i] > 0.1:
        
            df_300['study'][i] = 0
        
    else:
       
            df_300['study'][i] = 1
        
    

C:\Users\JUAN\Anaconda3\lib\site-packages\ipykernel\__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\JUAN\Anaconda3\lib\site-packages\ipykernel\__main__.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\JUAN\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2881: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  exec(code_obj, self.user_global_ns, self.user_ns)
C:\Users\JUAN\Anaconda3\lib\site-p

In [6]:
from sklearn.naive_bayes import MultinomialNB

#Valores que serão dados de cada musica(Vocal e Energia)
X = []
#Valores que serão entregues de 1(Musica boa para estudo) e 0(Musica ruim para estudo)
train_ans = []

for i in range(31):
    X.append([df_300['speechiness'][i],df_300['energy'][i]])
    train_ans.append(df_300['study'][i])
    
songs= MultinomialNB()
y = np.array(train_ans)
songs.fit(X=X,y=y)

#Usando o predict podemos tentar prever 5 valores de X
print(songs.predict(X[0:6]))

#Conferindo o predict com os valores reais
print(y[0:6])


#É preciso denotar que por terem sido usados apenas 30 valores para a calibragem, o fit nao ficará preciso.

[0 1 1 1 1 1]
[0 1 0 1 1 0]


In [150]:
df=df.iloc[:,:14] #retirando o nome da cançao e do artista
df["loudness"]*=-1 #transformando o atributo loudness em positivo
#df_train["study"].tolist() #passando o dataframe para uma lista

In [153]:

train30_array=np.array(df[:31].values.tolist()).astype(np.float64) #passando o dataframe para uma lista com valores de type=float
train30_chosen=df_train["study"].tolist() #passando a as musicas classificadas (base de treino)

train30=MultinomialNB()
train30.fit(train30_array,train30_chosen) #fit do database de treino com as musicas classificadas
study_songs=[]
non_study_songs=[]
maybe_songs=[]
for i in range(30): #algoritmo fazendo sua classificação de musicas novas
    if train30.predict(df[i+30:i+31])==2:
        study_songs.append(df_300["song_title"][i+30:i+31])
    elif train30.predict(df[i+30:i+31])==1:
        maybe_songs.append(df_300["song_title"][i+30:i+31])
    else:
        non_study_songs.append(df_300["song_title"][i+30:i+31])
study_songs

[43    Monster
 Name: song_title, dtype: object, 46    Fantastic Man
 Name: song_title, dtype: object, 49    Odofo Nyi Akyiri Biara
 Name: song_title, dtype: object, 53    Wishes
 Name: song_title, dtype: object, 59    California Nights
 Name: song_title, dtype: object]

In [152]:
non_study_songs

[30    Milly Rock
 Name: song_title, dtype: object, 31    Blasé (feat. Future & Rae Sremmurd)
 Name: song_title, dtype: object, 32    Best Friend
 Name: song_title, dtype: object, 33    Hercules
 Name: song_title, dtype: object, 34    Who Be Lovin Me (feat. ILOVEMAKONNEN)
 Name: song_title, dtype: object, 35    Check
 Name: song_title, dtype: object, 36    Poppin' My Collar
 Name: song_title, dtype: object, 37    Jumpman
 Name: song_title, dtype: object, 38    7/11
 Name: song_title, dtype: object, 39    Somebody's Watching Me
 Name: song_title, dtype: object, 40    Bang That
 Name: song_title, dtype: object, 41    I Don't Sell Molly No More
 Name: song_title, dtype: object, 42    Versace Python
 Name: song_title, dtype: object, 44    OOOUUU
 Name: song_title, dtype: object, 45    Return Of The Mack - C & J Street Mix
 Name: song_title, dtype: object, 47    Right Place Wrong Time
 Name: song_title, dtype: object, 48    Azon de ma gnin kpevi
 Name: song_title, dtype: object, 50    Annie

In [154]:
maybe_songs

[31    Blasé (feat. Future & Rae Sremmurd)
 Name: song_title, dtype: object, 40    Bang That
 Name: song_title, dtype: object]